# DNN 기반 회귀 모델 구현 + 학습 진행 모니터링 + 진행 파라미터 저장
- 데이터셋 : iris.csv
- 피쳐 : 3개 sepal_length, sepal_width, petal_length
- 타겟/라벨 : 1개 (petal_width)
- 학습 방법 : 지도학습 - 회귀 
- 알고리즘 : 인공신경망(ANN) -> MLP(Multi Layer Perceptron), DNN ( ) : 은닉층이 많은 구성 
- 프레임 워크 : Pytorch

============================================================================================
- 모니터링 
    - 기준 : 검증 데이터셋의 loss 또는  score
    - 평가 : 학습 데이터셋의 loss 또는 score와 비교해서 학습 중단여부 결정 
    - 저장 : 현재까지 진행된 모델의 파라미터(가중치, 절편) 저장 여부 또는 모델 전체 저장 

## [1] 모듈 로딩 및 데이터 준비
---

In [52]:
# 모델 
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim

# 성능지표 
from torchmetrics.regression import R2Score, MeanSquaredError
from torchinfo import summary

# 데이터 처리 및 시각화  
import pandas as pd 
import matplotlib.pyplot as plt 
from sklearn.preprocessing import * 

from sklearn.model_selection import train_test_split


In [53]:
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

In [54]:
# 활용 패키지 버전 체크 => 사용자 정의 함수로 구현하기 
print(f'torch v.{torch.__version__}')
print(f'pandas v.{pd.__version__}')

torch v.2.4.1
pandas v.2.0.3


In [55]:
# 데이터 로딩 
DATA_FILE = '../DATA/iris.csv'

# csv => DF
iris_df = pd.read_csv(DATA_FILE, usecols=[0,1,2,3])

iris_df.head()

,sepal.length,sepal.width,petal.length,petal.width
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


## [2] 모델 클래스 설계 및 정의 
- 클래스 목적 : iris 데이터를 학습 및 추론 목적 
- 클래스 이름 : iris_reg_model
- 부모 클래스 : nn.Module 
- 매개 변수 : 층 별 입출력 개수 고정하기 때문에 필요 x 
- 속성 / 필드 : features_df, target_df, n_rows, n_features (df만들 때 사용)
- 기능 / 역할 
    - _ _init_ _() : 모델 구조 생성 
    - forward : 순방향 학습 (오버라이딩(overriding조건 : 상속관계에서만 가능))
- 클래스 구조 
    - 입력층 : 입력 3개(피처)        출력 10개(퍼셉트론/뉴런 10개 존재)
    - 은닉층 : 입력 10개            출력 30개 (퍼셉트론/뉴런 30개 존재)
    - 출력층 : 입력 30개            출력 1개 (타겟값(너비값))

- 활성화 함수 
    - 클래스 형태 ==> nn.MSELoss , nn.ReLU => _ _init_ _() 메서드 
    - 함수 형태 => torch.nn.functional 아래에 => forward() 메서드 

일반적으로 입력층의 퍼셉트론을 크게해서 점차적으로 줄임

super()는 부모 부르는 것 

In [56]:
class iris_reg_model(nn.Module):
    
    # 모델 구조 구성 및 인스턴스 생성 및 메서드 
    def __init__(self):
        super().__init__()

        # 모델 구조 구성 
        self.in_layer = nn.Linear(3,10)
        self.hidden_layer = nn.Linear(10,30)
        self.out_layer = nn.Linear(30,1)

    # 순방향 학습 진행 메서드 
    def forward(self, input_data):

        # 입력층 
        y = self.in_layer(input_data) # f11w11 + f12w12 + f13w13 + b, ......., f101w101 + f102w102 + f103w103 + b
                                      # # f11w11 + f21w21 + f31w31 + b, ......., f110w110 + f210w210 + f310w310 + b (내 생각)
        y = F.relu(y)             # relu => y 값의 범위 : 0 <= y 

        # 은닉층 : 10개의 숫자 값(>=0)
        y = self.hidden_layer(y)  # f21w11 + f22w12 .... + f210w210 + b, ......., f230w201 + f230w202 ..... f230w210 + b
        # 데이터 1개 기준
        y = F.relu(y) 

        # 출력층 : 30개의 숫자 값(>=0)
        # self.out_layer(y)         # f31w31 + ...... f330w330 + b
        # 회귀라서 활성함수 사용 x -> 바로 return 

        return self.out_layer(y)


In [57]:
# 모델 인스턴스 생성 
model = iris_reg_model()
print(model)

iris_reg_model(
  (in_layer): Linear(in_features=3, out_features=10, bias=True)
  (hidden_layer): Linear(in_features=10, out_features=30, bias=True)
  (out_layer): Linear(in_features=30, out_features=1, bias=True)
)


In [58]:
# 모델 사용 메모리 정보 확인 

summary(model, input_size=(10,3)) # 뒤에는 피쳐 수 , 아이리스 데이터 10개 줌 

Layer (type:depth-idx)                   Output Shape              Param #
iris_reg_model                           [10, 1]                   --
├─Linear: 1-1                            [10, 10]                  40
├─Linear: 1-2                            [10, 30]                  330
├─Linear: 1-3                            [10, 1]                   31
Total params: 401
Trainable params: 401
Non-trainable params: 0
Total mult-adds (M): 0.00
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.01

## [3] 데이터셋 클래스 설계 및 정의
- 데이터셋 : iris.csv 
- 피쳐 개수 : 3개 
- 타겟 개수 : 1개 
- 클래스 이름 : iris_data_set
- 부모 클래스 : utils.data.Dataset 
- 속성 / 필드 : feature_df, target_df 
- 필수 메서드 
    - _ _init_ _(self) : 데이터셋 저장 및 전처리, 개발자가 필요한 속성 설정 
    - _ _len_ _(self) : 데이터의 개수 반환 
    - _ _ getItem_ _(self, index) : 특정 인덱스의 피쳐와 타겟 반환 

In [59]:
class iris_data_set(Dataset):
    def __init__(self, feature_df, target_df):
        self.feature_df = feature_df
        self.target_df = target_df
        self.n_rows = feature_df.shape[0]
        self.n_features = feature_df.shape[1]

    def __len__(self):
        return self.n_rows

    def __getitem__(self,index):
        
        # 텐서화 
        feature_ts = torch.FloatTensor(self.feature_df.iloc[index].values) # 시리즈라서 values() 사용해서 numpy -> tensor 
        target_ts = torch.FloatTensor(self.target_df.iloc[index].values)
                
        # 피쳐와 타겟 반환 
        return feature_ts, target_ts

In [60]:
# 데이터셋 인스턴스 생성 

feature_df = iris_df[iris_df.columns[:-1]] # 회귀에서 같은 차원으로 해야 하므로 2D 
target_df = iris_df[iris_df.columns[-1:]]  # 2D 

feature_df.shape, target_df.shape


((150, 3), (150, 1))

In [61]:
# 커스텀 데이터셋 인스턴스 생성 
iris_ds = iris_data_set(feature_df, target_df)

## [4] 학습 준비 
- 학습 횟수 : EPOCH (처음부터 끝까지 학습할 단위)
- 배치 크기 : BATCH_SIZE (한 번에 학습할 데이터셋 양)
- 위치 지정 : DEVICE (텐서 저장 및 실행 위치 (GPU, CPU))
- 학습률 : 가중치와 절편 업데이트 시 경사하강법으로 업데이트 간격 설정 0.001 ~ 0.

In [62]:
# 학습 진행 관련 설정 
EPOCH = 1 
BATCH_SIZE = 10 
BATCH_CNT = iris_df.shape[0] // BATCH_SIZE
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
LR = 0.001 # hyper-parameter : 업데이트 간격  

print(f'BATCH_CNT : {BATCH_CNT}')

BATCH_CNT : 15


- 인스턴스 / 객체 : 모델, 데이터셋, 최적화, 손실함수 ,(성능지표)  

In [63]:
# 모델 인스턴스 
model = iris_reg_model()

# 데이터셋 인스턴스 
iris_ds = iris_data_set(feature_df, target_df)

# 데이터로더 인스턴스 
iris_dl = DataLoader(iris_ds, batch_size=BATCH_SIZE)


In [64]:
# 모델 인스턴스 
model = iris_reg_model()

# 데이터셋 인스턴스 
x_train, x_test, y_train, y_test = train_test_split(feature_df, target_df, random_state = 1)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, random_state = 1)
print(f'{x_train.shape} {x_test.shape} {x_val.shape}')
print(f'{y_train.shape} {y_test.shape} {y_val.shape}')
print(f'{type(x_train)} {type(x_test)} {type(x_val)}')
# iris_ds = iris_data_set(x_train, y_train)

train_ds = iris_data_set(x_train, y_train)
val_ds = iris_data_set(x_val, y_val)
test_ds = iris_data_set(x_test, y_test)

# 데이터로더 인스턴스 
# iris_dl = DataLoader(iris_ds, batch_size=BATCH_SIZE)
train_dl = DataLoader(train_ds, batch_size=BATCH_SIZE)


(84, 3) (38, 3) (28, 3)
(84, 1) (38, 1) (28, 1)
<class 'pandas.core.frame.DataFrame'> <class 'pandas.core.frame.DataFrame'> <class 'pandas.core.frame.DataFrame'>


In [65]:
## [테스트] 데이터 로더 

for feature, target in iris_ds:
    print(feature.shape, target.shape)
    

torch.Size([3]) torch.Size([1])
torch.Size([3]) torch.Size([1])
torch.Size([3]) torch.Size([1])
torch.Size([3]) torch.Size([1])
torch.Size([3]) torch.Size([1])
torch.Size([3]) torch.Size([1])
torch.Size([3]) torch.Size([1])
torch.Size([3]) torch.Size([1])
torch.Size([3]) torch.Size([1])
torch.Size([3]) torch.Size([1])
torch.Size([3]) torch.Size([1])
torch.Size([3]) torch.Size([1])
torch.Size([3]) torch.Size([1])
torch.Size([3]) torch.Size([1])
torch.Size([3]) torch.Size([1])
torch.Size([3]) torch.Size([1])
torch.Size([3]) torch.Size([1])
torch.Size([3]) torch.Size([1])
torch.Size([3]) torch.Size([1])
torch.Size([3]) torch.Size([1])
torch.Size([3]) torch.Size([1])
torch.Size([3]) torch.Size([1])
torch.Size([3]) torch.Size([1])
torch.Size([3]) torch.Size([1])
torch.Size([3]) torch.Size([1])
torch.Size([3]) torch.Size([1])
torch.Size([3]) torch.Size([1])
torch.Size([3]) torch.Size([1])
torch.Size([3]) torch.Size([1])
torch.Size([3]) torch.Size([1])
torch.Size([3]) torch.Size([1])
torch.Si

In [66]:
# 최적화 인스턴스 => w, b 텐서 즉, model.parameters() 전달 
optimizer = optim.Adam(model.parameters(), lr=LR)

# 손실함수 인스턴스 => 회귀, MSE, MAE, RMSE
req_loss = nn.MSELoss()

## [5] 학습 진행 

- >>> 모델 저장 준비

In [67]:
# models 폴더 아래 프로젝트 폴더 아래 모델 파일 저장 
import os 

# 저장 경로 
SAVE_PATH = '../MODELS/iris/'  

# 저장 파일명 
SAVE_FILE = 'model_train_wbs.pth'

# 모델 구조 및 파라미터 모두 저장 파일명 
SAVE_MODEL = 'model_all.pth'

In [68]:
# 경로상 폴더 존재 여부 체크 
if not os.path.exists(SAVE_PATH):
    os.makedirs(SAVE_PATH)   # 폴더 / 폴더 / ... 하위 폴더까지 생성 

- >>> 학습 진행

In [69]:
## 학습의 효과 확인 손실값과 성능평가값 저장 필요 
LOSS_HISTORY, SCORE_HISTORY = [],[]
CNT = iris_ds.n_rows/BATCH_SIZE
print(f'CNT : {CNT}')

for epoch in range(EPOCH):

    # 학습 모드로 모델 설정 
    model.train()

    # 배치 크기 만큼 데이터 로딩해서 학습 진행 
    loss_total, score_total = 0,0
    for feature_ts, target_ts in iris_dl: # iris_dl -> train_dl 

        # 학습 진행 
        pre_y = model(feature_ts)

        # 손실 계산 
        loss = req_loss(pre_y, target_ts)
        loss_total += loss.item() # tensor 라서 item으로 값 넣어야 함 

        # 성능 평가 계산 
        score = R2Score()(pre_y, target_ts) 
        score_total += score.item() # tensor 라서 item으로 값 넣어야 함 

        # 최적화 진행 
        optimizer.zero_grad()       
        loss.backward()
        optimizer.step()

 
    # for문 다 돌면 1 epoch 종료 
    # 손실값과 성능평가값 저장 
    LOSS_HISTORY.append(loss_total/BATCH_CNT)
    SCORE_HISTORY.append(score_total/BATCH_CNT)

CNT : 15.0


training, 검증, test 용 loader를 따로 만들어서 쪼개서 돌려야 함 
- 데이터가 적어서 한 번에 다 돌려도 되면 따로 안 만들어도 ㅇ 

-------------------
+) 모니터링 추가


In [70]:
## 학습의 효과 확인 손실값과 성능평가값 저장 필요 
LOSS_HISTORY, SCORE_HISTORY = [[],[]],[[],[]]
CNT = iris_ds.n_rows/BATCH_SIZE
print(f'CNT : {CNT}')

# =================================================================================================================
# 학습 모니터링/스케줄링 설정 
# => LOSS_HISTORY, SCORE_HISTORY 활용 
# => 임계 기준 : 10번 
BREAK_CNT = 0
LIMIT_VALUE = 10 # 멈추는 기준 
# =================================================================================================================


for epoch in range(EPOCH):

    # 학습 모드로 모델 설정 
    model.train()

    # 배치 크기 만큼 데이터 로딩해서 학습 진행 
    loss_total, score_total = 0,0
    for feature_ts, target_ts in train_dl: # iris_dl -> train_dl 

        # 학습 진행 
        pre_y = model(feature_ts)

        # 손실 계산 
        loss = req_loss(pre_y, target_ts)
        loss_total += loss.item() # tensor 라서 item으로 값 넣어야 함 

        # 성능 평가 계산 
        score = R2Score()(pre_y, target_ts) 
        score_total += score.item() # tensor 라서 item으로 값 넣어야 함 

        # 최적화 진행 
        optimizer.zero_grad()       
        loss.backward()
        optimizer.step()

    # 에포크 당 검증기능 
    # 모델 검증 모드 설정 
    model.eval()
    # 검증한 결과를 저장해야 함 

    with torch.no_grad():
        # 검증 데이터셋 
        val_feature_ts = torch.FloatTensor(val_ds.feature_df.values) # values쓰면 array 됨 -> torch사용 -> torch됨 
        val_target_ts = torch.FloatTensor(val_ds.target_df.values)

        # 평가 
        pre_val =model(val_feature_ts)
        
        # 손실 
        loss_val = req_loss(pre_val, val_target_ts)

        # 성능 평가 
        score_val = R2Score()(pre_val, val_target_ts)


    # for문 다 돌면 1 epoch 종료 
    # 손실값과 성능평가값 저장 
    LOSS_HISTORY[0].append(loss_total/BATCH_CNT)
    SCORE_HISTORY[0].append(score_total/BATCH_CNT)

    LOSS_HISTORY[1].append(loss_val)
    SCORE_HISTORY[1].append(score_val)

    # =================================================================================================================

    # 학습 진행 모니터링/스케줄링 - 검증 DS 기준 
    # 아래 중 둘 중 하나만 하면 됨 - 두개 다 하면 cnt만 올라감 

    # validation loss 
    if len(LOSS_HISTORY[1]) >= 2:
        if LOSS_HISTORY[1][-1] >= LOSS_HISTORY[1][-2] : BREAK_CNT +=1 # LOSS_HISTORY[1][-1]  < LOSS_HISTORY[1][-2]이 정상 

    # # validation score - 일반적으로 많이 봄   
    # if len(SCORE_HISTORY[1]) >= 2: 
    #     if SCORE_HISTORY[1][-1] <= SCORE_HISTORY[1][-2] : BREAK_CNT +=1 

    # =================================================================================================================

    # 성능이 좋은 학습 가중치 저장 
    if len(SCORE_HISTORY[1]) >= 1: 
        
        # 첫 번째라서 무조건 모델 저장 
        torch.save(model.state_dict(), SAVE_PATH + SAVE_FILE)

        # 모델 전체 저장 
        torch.save(model, SAVE_PATH + SAVE_MODEL)

    else:
        if SCORE_HISTORY[1][-1] > max(SCORE_HISTORY[1][:-1]) : 
            torch.save(model.state_dict(), SAVE_PATH + SAVE_FILE) 


    # 학습 중단 여부 설정 
    # if BREAK_CNT >= 9:  # BREAK_CNT가 0부터 시작하면 >= 9, 1부터 시작하면 >10
    #     print('성능 및 손실 개선이 없어서 학습 중단')
    #     break

    if BREAK_CNT > LIMIT_VALUE:  # LIMIT_VALUE 변수 사용 
        print('성능 및 손실 개선이 없어서 {epoch} EPOCH에 학습 중단')
        break

CNT : 15.0


In [71]:
print(f'LOSS_HISTORY => ', LOSS_HISTORY )
print(f'SCORE_HISTORY => ', SCORE_HISTORY)

LOSS_HISTORY =>  [[0.5581796367963155], [tensor(0.5657)]]
SCORE_HISTORY =>  [[-0.6192238966623942], [tensor(0.1237)]]


### 모델 저장 방법 
---
- 방법 1 : 모델 파라미터만 저장 
- 방법 2 : 모델 설계 구조 및 파라미터까지 모두 저장

In [72]:
# 학습된 모델 파라미터 값 확인 
model.state_dict() 

OrderedDict([('in_layer.weight',
              tensor([[-0.0998,  0.2970,  0.4393],
                      [ 0.2866, -0.4380, -0.2823],
                      [-0.5130,  0.3374, -0.2216],
                      [ 0.2484, -0.2774,  0.0883],
                      [-0.5772, -0.2728, -0.2623],
                      [ 0.3190, -0.4263,  0.3648],
                      [-0.5419,  0.4057,  0.3879],
                      [-0.1081,  0.3137,  0.2954],
                      [ 0.1035, -0.3308,  0.2643],
                      [ 0.5025,  0.4958,  0.2419]])),
             ('in_layer.bias',
              tensor([-0.4043, -0.3055,  0.2027,  0.4172,  0.2207, -0.1202,  0.5360, -0.0634,
                      -0.2832,  0.3893])),
             ('hidden_layer.weight',
              tensor([[ 2.0388e-01, -4.1358e-02, -1.1087e-01,  2.3670e-01, -1.3656e-01,
                        2.0735e-01, -8.1007e-03, -2.2177e-01,  4.5139e-02, -2.7864e-01],
                      [ 1.9834e-01,  1.1930e-01, -1.3308e-01, -2.6889e-0

- [방법 1] 모델 파라미터 즉, 층별 가중치와 절편들

In [73]:
# 모델 즉, 가중치와 절편 로딩 
# [1] 가중치와 절편 객체로 로딩 
# [2] 모델의 state_dict 속성에 저장 

# 읽기 
wb_ts = torch.load(SAVE_PATH + SAVE_FILE,weights_only=True)
print(type(wb_ts))

<class 'collections.OrderedDict'>


In [74]:
# 모델 인스턴스에 저장 
model2 = iris_reg_model()        # 층마다 w,b 초기화 - 처음부터 다시 돌려야 함 
model2.load_state_dict(wb_ts) 

# 기존에 했던 것 까지는 저장됨 

<All keys matched successfully>

In [75]:
print(model2) # 모델 층, 퍼셉트론 수 동일해야 함  

iris_reg_model(
  (in_layer): Linear(in_features=3, out_features=10, bias=True)
  (hidden_layer): Linear(in_features=10, out_features=30, bias=True)
  (out_layer): Linear(in_features=30, out_features=1, bias=True)
)
